In [1]:
import warnings
import seaborn as sns
import matplotlib.pylab as plt
import PIL
import pandas as pd
import os

import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import Xception
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import layers, models, optimizers

from sklearn.model_selection import train_test_split

import numpy as np # linear algebra

from numpy.random import seed

Using TensorFlow backend.


In [2]:
warnings.filterwarnings('ignore')

SEED = 2
seed(SEED)
tf.set_random_seed(SEED)

In [3]:
print(os.listdir("../input"))

['test_crop', 'test', 'class.csv', 'test.zip', 'train.zip', 'sample_submission.csv', 'train', '.ipynb_checkpoints', 'train.csv', 'test.csv', 'train_crop']


In [4]:
DATA_PATH = '../input'
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')
TEST_IMG_PATH = os.path.join(DATA_PATH, 'test')
TRAIN_CROP_PATH = os.path.join(DATA_PATH, 'train_crop')
TEST_CROP_PATH = os.path.join(DATA_PATH, 'test_crop')

In [5]:
def crop_resize_boxing_img(img_name, margin=8, size=(299, 299)) :
    if img_name.split('_')[0] == "train" :
        PATH = TRAIN_IMG_PATH
        data = df_train
    elif img_name.split('_')[0] == "test" :
        PATH = TEST_IMG_PATH
        data = df_test
        
    img = PIL.Image.open(os.path.join(PATH, img_name))
    pos = data.loc[data["img_file"] == img_name, \
                   ['bbox_x1','bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1)

    width, height = img.size
    x1 = max(0, pos[0] - margin)
    y1 = max(0, pos[1] - margin)
    x2 = min(pos[2] + margin, width)
    y2 = min(pos[3] + margin, height)

    return img.crop((x1,y1,x2,y2)).resize(size)

In [6]:
def crop_img():
    if not os.path.exists(TRAIN_CROP_PATH):
        os.mkdir(TRAIN_CROP_PATH)
        print('>>mkdir {}'.format(TRAIN_CROP_PATH))
        for i, row in df_train.iterrows():
            cropped = crop_resize_boxing_img(row['img_file'])
            cropped.save(TRAIN_CROP_PATH+"/"+row['img_file'])
        print('>>train_crop completed')
    else:
        print('>>train_crop exist')
    if not os.path.exists(TEST_CROP_PATH):
        print('>>mkdir {}'.format(TEST_CROP_PATH))
        os.mkdir(TEST_CROP_PATH)
        for i, row in df_test.iterrows():
            cropped = crop_resize_boxing_img(row['img_file'])
            cropped.save(TEST_CROP_PATH+"/"+row['img_file'])
        print('>>test_crop  completed')
    else:
        print('>>test_crop exist')

In [145]:
def split_traindf(df, train_size=0.6, stratify=True, label='class'):
    target = None
    if stratify:
        target = df[label].values
    X_train, X_val = train_test_split(df, train_size=train_size, random_state=SEED, stratify=target)
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    return X_train, X_val

In [204]:
def get_generator(train_df, val_df, train_dir, valid_dir, test_df, test_dir, image_size, batch_size,valid_batch_size,
                 scale='rgb', target='class'):
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df, 
        directory=train_dir,
        x_col = 'img_file',
        y_col = target,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        seed=3,
        color_mode=scale,
        shuffle=True
    )
    validation_generator = valid_datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=valid_dir,
        x_col = 'img_file',
        y_col = target,
        target_size=(image_size,image_size),
        batch_size=valid_batch_size,
        class_mode='categorical',
        seed=3,
        color_mode=scale,
        shuffle=True
    )
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=test_dir,
        x_col='img_file',
        y_col=None,
        target_size= (image_size,image_size),
        color_mode=scale,
        class_mode=None,
        batch_size=batch_size,
        shuffle=False
    )
    return train_generator, validation_generator, test_generator



In [177]:
def get_model(num_class=196):
    application = Xception
    base_model = application(weights='imagenet', input_shape=(image_size,image_size,3), include_top=False)
    #base_model.trainable = False

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='sigmoid'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_class, activation='softmax'))
    #model.summary()
    optimizer = optimizers.RMSprop(lr=lr)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

    return model

In [10]:
def get_model_path(model_dir, model_name):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    from datetime import datetime
    now = datetime.now()
    date_time = now.strftime("%m%d_%H%M")
    model_path = model_dir + date_time + model_name + '.hdf5'
    print('>>model path to save: {}'.format(model_path))
    return model_path

In [11]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size
    

In [12]:
def plot_acc(history, model_name):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training Acc')
    plt.plot(epochs, val_acc, 'b', label='Validation Acc')
    plt.title('Training and validation accuracy' + model_name)
    plt.legend()
    plt.show()

    plt.plot(epochs, loss, 'bo', label='Traing loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Trainging and validation loss' + model_name)
    plt.legend()
    plt.show()


In [184]:
def get_class_name(df_train, df_class):
    df_class = df_class.copy()
    df_train = df_train.copy()
    import re
    p = re.compile('[A-Z]*[a-z]* ')
    new = [x[0] for x in df_class['name'].str.findall(p)]
    df_class['name'] = new
    df_class=df_class.rename(columns = {'id':'class'})
    df_class['class'] = df_class['class'].astype(str)
    df_train = pd.merge(df_train, df_class, on='class')
    return df_train

In [225]:
K.clear_session()

df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))

crop_img()

df_train["class"] = df_train["class"].astype('str')
df_train = df_train[['img_file', 'class']]
df_test = df_test[['img_file']]

>>train_crop exist
>>test_crop exist


## predict car brand

In [226]:
df_train = get_class_name(df_train, df_class)
indexs=[]
for name in df_train['name'].unique():
    max_val = 20
    label_idx = np.argwhere(df_train['name']== name)[:,0]
    np.random.shuffle(label_idx)
    indexs.extend(label_idx[:max_val])
df_train_brand = df_train.iloc[indexs,:]

In [227]:
train_datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [230]:
image_size = 299
batch_size = 4
valid_batch_size = 16
#X_train, X_val = split_traindf(df_train_brand.iloc[:, :], train_size=1.0, stratify=True, label='name')
X_val = df_train[~df_train.index.isin(indexs)]
X_train = df_train_brand
nb_train_sample = X_train.shape[0]
nb_validation_sample = X_val.shape[0]
nb_test_sample = df_test.shape[0]
print(nb_train_sample)
print(nb_validation_sample)
print(nb_test_sample)
scale = 'rgb'
#scale = 'rgb'
train_gen, validation_gen, test_gen = get_generator(train_df=X_train,
                                                    val_df=X_val,
                                                    train_dir=TRAIN_CROP_PATH,
                                                    valid_dir=TRAIN_CROP_PATH,
                                                    test_df=df_test,
                                                    test_dir=TEST_CROP_PATH,
                                                    image_size=image_size,
                                                    batch_size=batch_size,
                                                    valid_batch_size=valid_batch_size,
                                                    scale=scale,
                                                   target='name')


1470
8546
6169
Found 1470 validated image filenames belonging to 49 classes.
Found 8546 validated image filenames belonging to 49 classes.
Found 6169 validated image filenames.


In [233]:
histories=[]
patient = 3
lr = 0.00005
model_dir = '../xception_model/'
model_name = '_3rd_xception_brand_preidct'
epoch=300


In [234]:
model = get_model(num_class=49)
model_path = get_model_path(model_dir, model_name)

callbacks1 = [
    EarlyStopping(monitor='val_loss',
                  patience=patient,
                  mode='min',
                  verbose=1),
    #ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = patient / 2, min_lr=0.00001, verbose=1, mode='min'),
    ModelCheckpoint(filepath=model_path,
                    monitor='val_loss',
                    verbose=1,
                    save_best_only=True,
                    mode='min'),
]

history_brand = model.fit_generator(
    train_gen,
    steps_per_epoch=get_steps(nb_train_sample, batch_size),
    epochs=epoch,
    validation_data=validation_gen,
    validation_steps=get_steps(nb_validation_sample, batch_size),
    verbose=1,
    callbacks = callbacks1
)

>>model path to save: ../xception_model/0623_0526_3rd_xception_brand_preidct.hdf5
Epoch 1/300
368/368 [==============================] - 220s 597ms/step - loss: 4.1410 - acc: 0.0197 - val_loss: 3.8202 - val_acc: 0.0567

Epoch 00001: val_loss improved from inf to 3.82021, saving model to ../xception_model/0623_0526_3rd_xception_brand_preidct.hdf5
Epoch 2/300
368/368 [==============================] - 220s 598ms/step - loss: 3.8731 - acc: 0.0530 - val_loss: 3.5627 - val_acc: 0.1108

Epoch 00002: val_loss improved from 3.82021 to 3.56271, saving model to ../xception_model/0623_0526_3rd_xception_brand_preidct.hdf5
Epoch 3/300
368/368 [==============================] - 219s 596ms/step - loss: 3.5372 - acc: 0.1107 - val_loss: 3.4643 - val_acc: 0.1325

Epoch 00003: val_loss improved from 3.56271 to 3.46427, saving model to ../xception_model/0623_0526_3rd_xception_brand_preidct.hdf5
Epoch 4/300
368/368 [==============================] - 221s 601ms/step - loss: 3.2118 - acc: 0.1923 - val_loss: 

KeyboardInterrupt: 

Fail!!!

Unbalanced data. 30 ~ 80.
not enough to training.


In [ ]:
def train_each_models():
    brands = df_train_brand.unique()
    model_dict = {}
    for brand in brands:
        df = df_train[df_train['name']==brand]
        num_class = df['class'].unique()
        image_size = 299
        batch_size = 8

        X_train, X_val = split_traindf(df.iloc[:, :], train_size=0.7, stratify=True, label='class')

        nb_train_sample = X_train.shape[0]
        nb_validation_sample = X_val.shape[0]
        print(nb_train_sample)
        print(nb_validation_sample)
        scale = 'rgb'
        #scale = 'rgb'
        train_gen, validation_gen, test_gen = get_generator(train_df=X_train,
                                                            val_df=X_val,
                                                            train_dir=TRAIN_CROP_PATH,
                                                            valid_dir=TRAIN_CROP_PATH,
                                                            test_df=df_test,
                                                            test_dir=TEST_CROP_PATH,
                                                            image_size=image_size,
                                                            batch_size=batch_size,
                                                            scale=scale,
                                                            target='class')

        model = get_model(num_class=num_class)
        model_path = get_model_path('brand_model', brand)
        patient=3
        callbacks1 = [
            EarlyStopping(monitor='val_loss',
                          patience=patient,
                          mode='min',
                          verbose=1),
            #ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = patient / 2, min_lr=0.00001, verbose=1, mode='min'),
            ModelCheckpoint(filepath=model_path,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='min'),
        ]
        epoch=50
        history_brand = model.fit_generator(
            train_gen,
            steps_per_epoch=get_steps(nb_train_sample, batch_size),
            epochs=epoch,
            validation_data=validation_gen,
            validation_steps=get_steps(nb_validation_sample, batch_size),
            verbose=1,
            callbacks = callbacks1
        )
        
        model_dict.update({'history':history_brand})
        model_dict.update({'model':model})

In [ ]:
image_size = 299
batch_size = 16

X_train, X_val = split_traindf(df_train.iloc[:, :], train_size=0.7, stratify=True)
    

nb_train_sample = X_train.shape[0]
nb_validation_sample = X_val.shape[0]
nb_test_sample = df_test.shape[0]
scale = 'rgb'
#scale = 'rgb'
train_gen, validation_gen, test_gen = get_generator(train_df=X_train,
                                                    val_df=X_val,
                                                    train_dir=TRAIN_CROP_PATH,
                                                    valid_dir=TRAIN_CROP_PATH,
                                                    test_df=df_test,
                                                    test_dir=TEST_CROP_PATH,
                                                    image_size=image_size,
                                                    batch_size=batch_size,
                                                    scale=scale)


In [23]:
histories=[]
patient = 3
lr = 0.0005
model_dir = '../xception_model/'
model_name = '_3rd_xception_rmsprop_batch8'
epoch=300


In [24]:
model = get_model()
model_path = get_model_path(model_dir, model_name)

callbacks1 = [
    EarlyStopping(monitor='val_loss',
                  patience=patient,
                  mode='min',
                  verbose=1),
    #ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = patient / 2, min_lr=0.00001, verbose=1, mode='min'),
    ModelCheckpoint(filepath=model_path,
                    monitor='val_loss',
                    verbose=1,
                    save_best_only=True,
                    mode='min'),
]

history = model.fit_generator(
    train_gen,
    steps_per_epoch=get_steps(nb_train_sample, batch_size),
    epochs=epoch,
    validation_data=validation_gen,
    validation_steps=get_steps(nb_validation_sample, batch_size),
    verbose=1,
    callbacks = callbacks1
)

>>model path to save: ../xception_model/0623_0223_3rd_xception_rmsprop_batch8.hdf5
Epoch 1/300
877/877 [==============================] - 200s 229ms/step - loss: 5.0125 - acc: 0.0301 - val_loss: 4.0876 - val_acc: 0.0729

Epoch 00001: val_loss improved from inf to 4.08763, saving model to ../xception_model/0623_0223_3rd_xception_rmsprop_batch8.hdf5
Epoch 2/300
877/877 [==============================] - 196s 224ms/step - loss: 3.5755 - acc: 0.1365 - val_loss: 3.2075 - val_acc: 0.1983

Epoch 00002: val_loss improved from 4.08763 to 3.20749, saving model to ../xception_model/0623_0223_3rd_xception_rmsprop_batch8.hdf5
Epoch 3/300
877/877 [==============================] - 196s 224ms/step - loss: 2.5894 - acc: 0.3051 - val_loss: 2.1717 - val_acc: 0.4166

Epoch 00003: val_loss improved from 3.20749 to 2.17167, saving model to ../xception_model/0623_0223_3rd_xception_rmsprop_batch8.hdf5
Epoch 4/300
877/877 [==============================] - 196s 224ms/step - loss: 1.8829 - acc: 0.4629 - val_lo

KeyboardInterrupt: 

In [ ]:
plot_acc(history, '_xception')

In [ ]:
def make_submission(model, model_path, test_gen, model_name, nb_test_sample, batch_size):
    
    model.load_weights(model_path)
    test_gen.reset()
    prediction = model.predict_generator(
        generator=test_gen,
        steps = get_steps(nb_test_sample, batch_size),
        verbose=1
    )
    
    predicted_class_indices=np.argmax(prediction, axis=1)

    # Generator class dictionary mapping
    
    labels = (train_generator.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    predictions = [labels[k] for k in predicted_class_indices]

    submission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
    submission["class"] = predictions
    submission_fname = "submissions/submissions{}.csv".format(model_name)
    submission.to_csv(submission_fname, index=False)
    submission.head()
    print('[*]sumission saved at {}'format(submission_fname))
                      
make_submission(model, model_path, test_gen, model_name, nb_test_sample, batch_size)

base : batch 8 + more augmentation  + nodecay  + imgsize (224->299)

rmsprop ->

adam -> 